Notebook purpose

- Replicate analysis of davenport2020spending

In [99]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
d = aws.S3BucketManager("")
d.list()

['3di-data-experian',
 '3di-data-fable',
 '3di-data-huq',
 '3di-data-mdb',
 '3di-data-ons',
 '3di-project-entropy',
 '3di-project-habits',
 '3di-project-te']

In [320]:
df = aws.s3read_parquet('s3://3di-data-mdb/raw/mdb_000.parquet')
df.head(2)

,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,686979264,564777,2020-01-18,1986.0,NaN,CF83 1,W01001427,W02000313,F,2018-10-15,1564118,Lloyds Bank,Current,171.960007,nespresso uk ltd,Debit,15.600000,"Food, Groceries, Household",No Tag,"Food, Groceries, Household",Nespresso,Nespresso,2020-01-18,2020-04-17 00:10:00,2020-01-19,1900-01-01,C
1,335712120,299777,2015-05-10,1970.0,NaN,BS11 0,E01014498,E02003014,M,2018-04-05,427663,Santander,Current,0.000000,"direct debit payment to paypal payment ref 5ae22226hzlca, mandate no 0016",Debit,34.990002,Enjoyment,No Tag,Enjoyment,PayPal,PayPal,2015-05-10,2018-09-19 19:58:00,2018-04-07,1900-01-01,C


In [321]:
df = aws.s3read_parquet('s3://3di-project-entropy/entropy_000.parquet')
df.head(2)

,id,date,user_id,amount,desc,merchant,tag_up,user_female,user_lsoa,user_msoa,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,credit_debit,data_warehouse_date_created,data_warehouse_date_last_updated,latest_recorded_balance,merchant_business_line,tag_auto,tag_manual,updated_flag
0,688283,2012-01-17,777,42.410000,"9572 16jan12 , esso altrincham , sstn , altrincham gb - pos",Esso,Fuel,False,E01012553,E02002603,WA1 4,2011-07-20,NaN,1969.0,2011-07-20,262916,2020-06-25 22:55:00,NatWest Bank,Current,Debit,2014-07-18,2017-08-15,2378.280029,Esso,Fuel,NaN,U
1,248375,2012-07-23,777,39.389999,"9572 20jul12 , sacat marks & , spencer , gemini gb , cash back 30.00 - pos",NaN,Groceries,False,E01012553,E02002603,WA1 4,2011-07-20,NaN,1969.0,2011-07-20,262916,2020-06-25 22:55:00,NatWest Bank,Current,Debit,2014-07-18,2017-10-23,2378.280029,Personal,Cash,Groceries,U


In [336]:

cols = ['tag_up', 'tag_auto', 'tag_manual', 'merchant', 'merchant business line']
cols

['tag_up', 'tag_auto', 'tag_manual', 'merchant', 'merchant business line']

In [327]:
df

,tag_up,tag_auto,tag_manual
0,Fuel,Fuel,NaN
1,Groceries,Cash,Groceries
2,Interest charges,Interest charges,NaN
3,Vehicle insurance,Insurance,NaN
4,Dining and drinking,Dining or Going Out,NaN
...,...,...,...
995,NaN,NaN,NaN
996,Dining and drinking,Dining and drinking,NaN
997,"Medical, Dental, Eye Care","Medical, Dental, Eye Care",NaN
998,NaN,NaN,NaN


In [326]:
df[df.merchant_business_line.str.contains('Unkno')].merchant_business_line.value_counts()

Unknown Merchant    158
Peacocks              0
Pizza Express         0
Plusnet               0
Poundland             0
                   ... 
Greater Anglia        0
Greggs                0
Groupon               0
H&M                   0
trainline.com         0
Name: merchant_business_line, Length: 178, dtype: int64

In [302]:
df

,id,date,user_id,amount,desc,merchant,tag_up,user registration date,user_female,user_lsoa,user_msoa,user_postcode,user_salary_range,user_yob,account last refreshed,account type,account_created,account_id,account_provider,credit debit,data warehouse date created,data warehouse date last updated,latest recorded balance,merchant business line,tag_auto,tag_manual,updated_flag
0,688283,2012-01-17,777,42.410000,"9572 16jan12 , esso altrincham , sstn , altrincham gb - pos",Esso,Fuel,2011-07-20,False,E01012553,E02002603,WA1 4,NaN,1969.0,2020-06-25 22:55:00,Current,2011-07-20,262916,NatWest Bank,Debit,2014-07-18,2017-08-15,2378.280029,Esso,Fuel,No Tag,U
1,248375,2012-07-23,777,39.389999,"9572 20jul12 , sacat marks & , spencer , gemini gb , cash back 30.00 - pos",No Merchant,Groceries,2011-07-20,False,E01012553,E02002603,WA1 4,NaN,1969.0,2020-06-25 22:55:00,Current,2011-07-20,262916,NatWest Bank,Debit,2014-07-18,2017-10-23,2378.280029,Personal,Cash,Groceries,U
2,6755827,2013-06-21,777,7.410000,02jun a/c xxxx5781 - int,No Merchant,Interest charges,2011-07-20,False,E01012553,E02002603,WA1 4,NaN,1969.0,2020-06-25 22:55:00,Current,2011-07-20,262916,NatWest Bank,Debit,2014-07-18,2017-08-14,2378.280029,Account Provider,Interest charges,No Tag,U
3,10715048,2013-11-08,777,45.970001,ail quote me happy - d/d,Aviva,Vehicle insurance,2011-07-20,False,E01012553,E02002603,WA1 4,NaN,1969.0,2020-06-25 22:55:00,Current,2011-07-20,262916,NatWest Bank,Debit,2014-07-18,2018-04-30,2378.280029,Aviva,Insurance,No Tag,U
4,14002498,2014-02-03,777,11.780000,"9572 02feb14 , kfc warrington , warrington gb - pos",KFC,Dining and drinking,2011-07-20,False,E01012553,E02002603,WA1 4,NaN,1969.0,2020-06-25 22:55:00,Current,2011-07-20,262916,NatWest Bank,Debit,2014-07-18,2017-08-13,2378.280029,KFC,Dining or Going Out,No Tag,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,787160371,2020-02-17,586777,3.100000,icafe kelvingrove,No Merchant,No Tag,2020-06-09,False,S01009880,S02001863,G41 5,NaN,1989.0,2020-06-25 13:20:00,Current,2020-06-10,1705750,Bank of Scotland (BoS),Debit,2020-06-10,1900-01-01,220.899994,Unknown Merchant,No Tag,No Tag,C
996,787096708,2020-03-02,586777,0.990000,mcdonalds,McDonalds,Dining and drinking,2020-06-09,False,S01009880,S02001863,G41 5,NaN,1989.0,2020-06-25 13:20:00,Current,2020-06-10,1705750,Bank of Scotland (BoS),Debit,2020-06-10,1900-01-01,220.899994,McDonalds,Dining and drinking,No Tag,C
997,790872375,2018-02-22,587777,11.050000,church pharmacy on 21 feb clp,No Merchant,"Medical, Dental, Eye Care",2020-06-19,False,E01001253,E02000267,W7 3,NaN,1973.0,2020-06-25 03:11:00,Current,2020-06-19,1711781,Barclays,Debit,2020-06-20,1900-01-01,7974.080078,No Merchant Business Line,"Medical, Dental, Eye Care",No Tag,C
998,790872260,2018-07-04,587777,3.010000,<mdbremoved> bgc,No Merchant,No Tag,2020-06-19,False,E01001253,E02000267,W7 3,NaN,1973.0,2020-06-25 03:11:00,Current,2020-06-19,1711781,Barclays,Credit,2020-06-20,1900-01-01,7974.080078,No Merchant Business Line,No Tag,No Tag,C


In [244]:
df = pd.DataFrame({'A': ['Fuel', 'batman', 'bait'],
                    'B': ['abc', 'bar', 'xyz']})
# df.replace(regex=[r'^ba.$', 'xyz'], value='new')
df

,A,B
0,Fuel,abc
1,batman,bar
2,bait,xyz


In [247]:
regex = r'F\w+'

df.replace(regex=regex, value='new')

,A,B
0,new,abc
1,batman,bar
2,bait,xyz


In [248]:
# dd = df[cols]
dd.replace(regex=regex, value='new')

,tag_up,tag_auto,tag_manual,merchant,merchant business line
0,Fuel,Fuel,No Tag,Esso,Esso
1,Groceries,Cash,Groceries,No Merchant,Personal
2,Interest charges,Interest charges,No Tag,No Merchant,Account Provider
3,Vehicle insurance,Insurance,No Tag,Aviva,Aviva
4,Dining and drinking,Dining or Going Out,No Tag,KFC,KFC
...,...,...,...,...,...
995,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant
996,Dining and drinking,Dining and drinking,No Tag,McDonalds,McDonalds
997,"Medical, Dental, Eye Care","Medical, Dental, Eye Care",No Tag,No Merchant,No Merchant Business Line
998,No Tag,No Tag,No Tag,No Merchant,No Merchant Business Line


In [47]:
df.user_female.value_counts(dropna=False)

False    583
True     417
Name: user_female, dtype: int64

In [78]:
df.merchant.replace({'No Merchant': None})

0              Esso
1               NaN
2               NaN
3             Aviva
4               KFC
           ...     
995             NaN
996       McDonalds
997             NaN
998             NaN
999    Virgin Media
Name: merchant, Length: 1000, dtype: category
Categories (167, object): ['AA', 'ASOS', 'Abel & Cole', 'Affinity Water', ..., 'Zizzi', 'giffgaff', 'myprotein.com', 'trainline.com']

In [41]:
df.drop(columns='id').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date                              1000 non-null   datetime64[ns]
 1   user_id                           1000 non-null   int32         
 2   amount                            1000 non-null   float32       
 3   desc                              1000 non-null   category      
 4   merchant                          1000 non-null   category      
 5   tag_up                            1000 non-null   category      
 6   user registration date            1000 non-null   datetime64[ns]
 7   user_gender                       1000 non-null   category      
 8   user_lsoa                         993 non-null    category      
 9   user_msoa                         993 non-null    category      
 10  user_postcode                     1000 non-null  

# Prepare data

In [6]:
def read_sample(sample):
    """Read clean data for sample."""
    fp = f's3://3di-data-mdb/clean/mdb_{SAMPLE}.parquet'
    return aws.s3read_parquet(fp)

def clean_data(df):
    """Prepare data for analysis"""
    return df


SAMPLE = '777'

df = (
    read_sample(SAMPLE)
    .pipe(clean_data)
)

print(df.shape)
df.head(2)

(682656, 28)


,id,date,user_id,amount,desc,merchant,tag_up,user_female,user_lsoa,user_msoa,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,credit_debit,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_recorded_balance,merchant_business_line,tag_auto,tag_manual,updated_flag
0,688261,2012-01-03,777,-400.00,<mdbremoved> - s/o,no merchant,other account,False,e01012553,e02002603,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,debit,2014-07-18,2017-11-13,True,364.22,non merchant mbl,transfers,other account,u
1,688264,2012-01-03,777,-10.27,"9572 30dec11 , mcdonalds , restaurant , winwick road gb - pos",mcdonalds,dining and drinking,False,e01012553,e02002603,wa1 4,2011-07-20,20k to 30k,1969.0,2011-07-20,262916,2020-07-21 20:32:00,natwest bank,current,debit,2014-07-18,2015-03-19,True,364.22,mcdonalds,dining and drinking,no tag,u


In [11]:
df.groupby('user_id').user_id.count().describe()

count      283.000000
mean      2412.212014
std       2341.533982
min          5.000000
25%        698.000000
50%       1610.000000
75%       3463.000000
max      13176.000000
Name: user_id, dtype: float64

In [7]:
df.user_id.nunique()

283

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682656 entries, 0 to 682655
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   id                                682656 non-null  int32         
 1   date                              682656 non-null  datetime64[ns]
 2   user_id                           682656 non-null  int32         
 3   amount                            682656 non-null  float32       
 4   desc                              682442 non-null  object        
 5   merchant                          682656 non-null  object        
 6   tag_up                            682656 non-null  object        
 7   user_female                       682656 non-null  bool          
 8   user_lsoa                         678394 non-null  object        
 9   user_msoa                         678394 non-null  object        
 10  user_postcode                   

# Exploration

In [36]:
df.date.describe()

/var/folders/xg/n9p73cf50s52twlnz7z778vr0000gn/T/ipykernel_2430/1371252037.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.date.describe()


count                  682656
unique                   3002
top       2018-04-03 00:00:00
freq                     1940
first     2012-01-03 00:00:00
last      2020-08-16 00:00:00
Name: date, dtype: object

In [28]:
df.tag_up.value_counts()[:5]

no tag                        188717
food, groceries, household     61572
transfers                      44864
cash                           29073
enjoyment                      27999
Name: tag_up, dtype: int64

In [35]:
mask = ((df.tag_up == 'transfers')
        & (df.account_type == 'savings')
        & (df.amount > 0))

df[mask].sort_values(['user_id', 'date'])[['user_id', 'date', 'amount', 'desc']][:100]

,user_id,date,amount,desc
6108,777,2019-02-11,120.000000,from a/c xxxx5781 via mobile - xfer - dpc
6109,777,2019-02-11,60.000000,from a/c xxxx5781 via mobile - xfer - dpc
6141,777,2019-02-25,250.000000,from a/c xxxx5781 via mobile - xfer - dpc
6142,777,2019-02-25,100.000000,from a/c xxxx5781 via mobile - xfer - dpc
6156,777,2019-02-27,100.000000,from a/c xxxx5781 via mobile - xfer - dpc
6252,777,2019-03-29,155.000000,from a/c xxxx5781 via mobile - xfer - dpc
6253,777,2019-03-29,175.000000,from a/c xxxx5781 via mobile - xfer - dpc
6458,777,2019-06-03,600.000000,<mdbremoved>
6504,777,2019-06-18,480.000000,<mdbremoved>
6607,777,2019-07-25,100.000000,from a/c xxxx5781 via mobile - xfer - dpc
